# Inferential statistics II - Bootstrapping

## Introduction

In the previous exercises you performed frequentist calculations to perform inference from a sample of data. Such inference relies on theory largely developed from the 19th Century onwards that is subject to certain assumptions or theoretical limits. These are fine if those assumptions hold in your case, and what you want to do has a known theoretical distribution, for example the mean of a sampling distribution that we looked at.

In these exercises we are once again going to load in the medical charge data but this time we are going to make inferences about the population using bootstrapping, that is simulating repeated re-runs of an experiment. If frequentism is about using assumptions and theoretical results to calculate what we expect to happen were an experiment to be run again and again and again, then bootstrapping is about using computing power to essentially re-run the sample draw again and again and again to see what actually happens.

## Prerequisites

Whilst these exercises do not strictly depend on these concepts, before working through them, you should have completed the preceding assignment and have a good understanding of such frequentist concepts as
* the _z_-statistic
* the _t_-statistic
* the difference and relationship between the two
* the Central Limit Theorem, its assumptions and consequences
* how to estimate the population mean and standard deviation from a sample
* the concept of a sampling distribution of a test statistic, particularly for the mean
* how to combine these concepts to calculate confidence intervals and p-values
* how those confidence intervals and p-values allow you to perform hypothesis (or A/B) tests

To complete these exercises, you should have covered the material on how to calculate bootstrap replicates of summary statistics. Having an appreciation of what confidence intervals and p-values are will be helpful.

In [1]:
import pandas as pd
import numpy as np
from numpy.random import seed
import matplotlib.pyplot as plt

## Medical charge data set

In [2]:
med_charges = pd.read_csv('data/insurance2.csv')

In [3]:
med_charges.head()

,age,sex,bmi,children,smoker,region,charges,insuranceclaim
0,19,0,27.900,0,1,3,16884.92400,1
1,18,1,33.770,1,0,2,1725.55230,1
2,28,1,33.000,3,0,2,4449.46200,0
3,33,1,22.705,0,0,1,21984.47061,0
4,32,1,28.880,0,0,1,3866.85520,1


In the previous assignment, you used the frequentist approach to estimate the lower limit for the 95% confidence interval on the mean hospital charge. This approach relies on statistical theory that has been developed over the years and is also limited to statistics for which theoretical results on the sampling distribution exist. These results are remarkably useful and applicable much of the time and under a surprisingly wide range of conditions.

Having calculated the 95% lower confidence interval using frequentist theory in the previous exercise, you would like to verify your calculations and check you get consistent results without making the assumptions required before. After all, the distribution of charges really was very non-normal.

__Q:__ Use bootstrap sampling to estimate the same 95% confidence interval lower limit as before.

__A:__

In [4]:
# FUNCTIONS
def bootstrap_replicate_1d(data, func):
    """Generate bootstrap replicate"""
    return func(np.random.choice(data, size=len(data)))


def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""
    
    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

In [5]:
np.random.seed(47)
N_rep = 10000
charges = med_charges.charges

# Calculate mean and standard deviation of charges: mean, std
mean = np.mean(charges)
std = np.std(charges, ddof = 1)

# Draw 10000 bootstrap replicates: bs_reps
bs_reps = draw_bs_reps(charges, np.mean, size=N_rep)

# Determine the lower confidence interval
lower_CI = np.percentile(bs_reps, 2.5)

print("The lower confidence interval is: ", lower_CI)

The lower confidence interval is:  12624.862119128531


If you performed 10000 replicates immediately after setting the random seed to 47, you should get the value 12724 here, which compares very well with the value 12725 obtained using the _t_-distribution confidence interval previously. It is a most pleasant result to see the predictions of classical frequentist theory match with results that are now possible through the number-crunching ability of computers.

Remember in the previous exercise set, we saw there are two ways of performing a _t_-test from a sample, depending on whether we can assume the groups have equal variance or not. We can actually easily test this using the bootstrap!

__Q:__ Calculate the 95% confidence interval for the difference between the standard deviations of insurance and non-insurance claim charges (insured - non-insured). Calculate the differences over 10000 replicates. Plot the histogram of values and mark the locations of the percentiles. State the null and alternative hypothesis and comment on whether you would retain or reject the null hypothesis in this case and why.

__A:__

$H_o: x_{insurance} - x_{none} = 0$

$H_a: x_{insurance} - x_{none} \neq 0$

In [6]:
# Separate insured vs non insured claims: y_insure, n_insure
y_insure = med_charges.charges[med_charges.insuranceclaim == 1]
n_insure = med_charges.charges[med_charges.insuranceclaim == 0]

np.random.seed(47)

# Draw bootstrap replicates for each set of claims: bs_reps_y, bs_reps_n
bs_reps_y = draw_bs_reps(y_insure, np.mean, size=10000)
bs_reps_n = draw_bs_reps(n_insure, np.mean, size=10000)

# Get the difference of means for each set of replicates: bs_reps
bs_reps = bs_reps_y - bs_reps_n

# Determine the confidence interval: CI
CI = np.percentile(bs_reps, [2.5, 97.5])

print('Difference of sample means: ', empirical_diff)
print('Confidence Interval: ', CI)

NameError: name 'empirical_diff' is not defined

In [ ]:
# Plot the data in a histogram
_ = plt.hist(bs_reps, bins=10)
_ = plt.axvline(CI[0], color= 'r', linestyle = '-.')
_ = plt.axvline(CI[1], color= 'r', linestyle = '-.')
_ = plt.axvline(np.mean(bs_reps), color = 'r')
_ = plt.title('Dist. of diff. between sample means: insurance v. no insurance')
_ = plt.xlabel('Cost')
_ = plt.ylabel('Number of Incidents')

---

## Confidence interval and p-value

The confidence interval above is often a useful quantity to estimate. If we wish to limit our expected probability of making a Type II error (where we wrongly reject the null hypothesis, and it is, instead, true) to $\alpha$, the associated confidence interval is our estimate of the interval within which we expect the true population value to be found $100\times(1 - \alpha)$% of the time we do this test. In the above we performed bootstrap replicates to estimate the interval and reject the null hypothesis if this interval did not contain zero. You will sometimes see such an interval reported in the output of statistical functions.

The partner of the confidence interval is the p-value. The p-value and the confidence interval are linked through our choice of $\alpha$. The p-value tells us how likely it is, under the null hypothesis, to get an outcome at least as extreme as what was observed. If this fails to reach the level of our _pre-specified_ $\alpha$, we decide the null hypothesis is sufficiently unlikely to be true and thus reject it. To calculate this p-value via the bootstrap, we have to put ourselves in a position where we are simulating the null hypothesis being true and then calculate the fraction of times we observe a result at least as extreme as that actually observed.

Remember how, previously, you used the _t_-test to calculate the p-value for the observed difference between the means of insured and non-insured medical cases. We're now going to repeat this but using the bootstrap.

__Q:__ Perform a bootstrapped hypothesis test at the 5% significance level ($\alpha = 0.05$) to calculate the p-value of the observed difference between insurance and non-insurance charges, state your null and alternative hypotheses and whether you retain or reject the null hypothesis for the given significance level.

__A:__ The hypotheses are:

$H_o: x_{insurance} - x_{none} = 0$

$H_a: x_{insurance} - x_{none} \neq 0$

with a significance level of $\alpha = 0.05$

----

In [ ]:
# FUNCTIONS
def diff_of_means(data1, data2):
    """Difference in means of two arrays."""
    
    # The difference of means of data1, data2: diff
    diff = (np.mean(data1) - np.mean(data2))
    return diff


In [ ]:
# Calculate difference of means: empirical_diff
empirical_diff = diff_of_means(y_insure, n_insure)

# Shift arrays: y_insure_shifted, n_insure_shifted
y_insure_shifted = y_insure - np.mean(y_insure) + mean
n_insure_shifted = n_insure - np.mean(n_insure) + mean

np.random.seed(47)

# Calculate bootstrap replicates: bs_reps_y, bs_reps_n
bs_reps_y = draw_bs_reps(y_insure_shifted, np.mean, size=10000)
bs_reps_n = draw_bs_reps(n_insure_shifted, np.mean, size=10000)

# Calcuate difference of means for each set of bootstrap replicates: bs_reps
bs_reps = bs_reps_y - bs_reps_n

# Calculate p-value
p = np.sum(bs_reps >= empirical_diff) / len(bs_reps)
print('The p-value is: ', p)

In [ ]:
mean

In [ ]:
n_insure_shifted

In [ ]:
y_insure_shifted

In [ ]:
empirical_diff

We reject our null hypothesis because our p-value is less than our significance value of .05.

__Q:__ To put the above result in perspective, plot the histogram of your bootstrapped differences along with lines marking the locations of the observed difference. (Why would we plot more than one line, given that we only have one observed difference?)

__A:__

In [ ]:
_ = plt.hist(bs_reps, bins=10)
_ = plt.axvline(empirical_diff, color = 'r', alpha = 0.5)
_ = plt.axvline(np.mean(bs_reps), color ='y', alpha = 0.5, linestyle= '-.')
_ = plt.title('Dist. of diff. between sample means: insurance v. no insurance')
_ = plt.xlabel('Cost')
_ = plt.ylabel('Number of Incidents')

__Q:__ Compare your p-value above with that obtained using the _t_-test function in the previous assignment. Do you think you would want to try to perform enough bootstrap replicates to observe a random difference as large as that we did observe?

__A:__ The p-value from before told us that there was a significant value but it was something we could not accept because the distribution was not normal. Even if we performed more bootstrap replicates we wouldn't be able to get the same results.

__Q:__ Remember the two variants of the _t_-test we performed in the previous assignment. Which one would you use now?

__A:__ Out of the two the second one is better because it accounts for different variances, but I don't think we could use either because they are not the appropriate tests in this situation.

__Q:__ If, instead of being asked whether the means of two groups were different, you were working with an ad-recommender team who wanted to release a new recommendation algorithm, and you were looking at click-through rate both for the current algorithm (call it A) and from trials of their new algorithm (call it B), would you perform a two-sided test as above? What would be your null and alternative hypotheses and what would be the real world consequence of rejecting the null hypothesis?

__A:__ I would perform a one sided test because I only care to find out if the click through rate is better with our new algorithm B. Our null hypothesis would be that the click through rate for A is equal to B. Our alternate hypothesis would be the click through rate for A is less than the click through rate of B. Rejecting the null hypothesis would mean that implementing our new algorithm would result in a better recommendation algorithm and in turn a better click through rate. 

# Learning outcomes

You've previously applied frequentist methods to calculate confidence intervals, p-values, and perform hypothesis tests. Frequentist methods use theoretical results to calculate what we expect would happen if experiments were to be run again and again and again. Now you've seen how you can do the same things using the bootstrap approach, which does not rely on such theory, and attendant assumptions, but instead literally does run experiments again and again and again.

In these exercises, you have
* calculated the same confidence interval lower limit as you did previously
* tested the assumption that the variances of the two groups (insured vs. non-insured) were equal - something a bit harder to do using the frequentist method because of the nature of the sampling distribution for variance
* calculated the p-value for the difference between the means of the two groups and compared with the result obtained using the previous frequentist approach

You are now well equipped to apply the bootstrap approach to a wide variety of problems. Just think about what conditions you wish to recreate in your simulated reruns.